In [1]:
!pip install scikit-learn


In [2]:
import os
import numpy as np

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Trainingdata') 

# Actions that we try to detect
actions = np.array(['Running', 'Walking', 'Still', 'Jumping'])

# Thirty videos worth of data
no_sequences = 20

# Videos are going to be 30 frames in length
sequence_length = 80



In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

2023-09-24 20:42:28.197749: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-24 20:42:28.338806: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-24 20:42:28.829278: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2023-09-24 20:42:28.829358: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot 

In [4]:
label_map = {label:num for num, label in enumerate(actions)}

In [5]:
label_map

{'Running': 0, 'Walking': 1, 'Still': 2, 'Jumping': 3}

In [6]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [7]:
np.array(sequences).shape

(80, 80, 132)

In [8]:
np.array(labels).shape

(80,)

In [9]:
X = np.array(sequences)

In [10]:
X.shape

(80, 80, 132)

In [11]:
y = to_categorical(labels).astype(int)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [13]:
y_train.shape

(76, 4)

In [14]:
X_train.shape

(76, 80, 132)

In [15]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard



In [16]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [17]:

model = Sequential()
model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(80,132)))

model.add(LSTM(256, return_sequences=True, activation='relu'))

model.add(LSTM(256, return_sequences=False, activation='relu'))


model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))
model.summary()

2023-09-24 20:43:05.153287: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-24 20:43:05.175027: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-24 20:43:05.175202: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-24 20:43:05.175846: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 80, 128)           133632    
                                                                 
 lstm_1 (LSTM)               (None, 80, 256)           394240    
                                                                 
 lstm_2 (LSTM)               (None, 256)               525312    
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 4)                 2

In [18]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(X_train, y_train, epochs=2000,callbacks=[tb_callback])


Epoch 1/2000


2023-09-24 20:43:17.876017: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


3/3 [==============================] - 4s 161ms/step - loss: 1.3835 - categorical_accuracy: 0.3026
Epoch 2/2000
3/3 [==============================] - 0s 156ms/step - loss: 1.3428 - categorical_accuracy: 0.3553
Epoch 3/2000
3/3 [==============================] - 0s 161ms/step - loss: 246.1360 - categorical_accuracy: 0.2237
Epoch 4/2000
3/3 [==============================] - 0s 162ms/step - loss: 213.1241 - categorical_accuracy: 0.2763
Epoch 5/2000
3/3 [==============================] - 0s 164ms/step - loss: 1.3692 - categorical_accuracy: 0.4211
Epoch 6/2000
3/3 [==============================] - 0s 155ms/step - loss: 1.3816 - categorical_accuracy: 0.2500
Epoch 7/2000
3/3 [==============================] - 0s 161ms/step - loss: 1.3857 - categorical_accuracy: 0.2632
Epoch 8/2000
3/3 [==============================] - 0s 157ms/step - loss: 1.3859 - categorical_accuracy: 0.2632
Epoch 9/2000
3/3 [==============================] - 0s 157ms/step - loss: 1.3840 - categorical_accuracy: 0.2632
E

3/3 [==============================] - 0s 162ms/step - loss: 1.1943 - categorical_accuracy: 0.5000
Epoch 74/2000
3/3 [==============================] - 0s 155ms/step - loss: 1.1867 - categorical_accuracy: 0.5000
Epoch 75/2000
3/3 [==============================] - 0s 149ms/step - loss: 1.1760 - categorical_accuracy: 0.5000
Epoch 76/2000
3/3 [==============================] - 0s 155ms/step - loss: 1.1605 - categorical_accuracy: 0.5395
Epoch 77/2000
3/3 [==============================] - 0s 166ms/step - loss: 1.1371 - categorical_accuracy: 0.6184
Epoch 78/2000
3/3 [==============================] - 0s 156ms/step - loss: 1.1181 - categorical_accuracy: 0.6579
Epoch 79/2000
3/3 [==============================] - 0s 157ms/step - loss: 1.0789 - categorical_accuracy: 0.6711
Epoch 80/2000
3/3 [==============================] - 0s 157ms/step - loss: 1.0433 - categorical_accuracy: 0.6579
Epoch 81/2000
3/3 [==============================] - 0s 155ms/step - loss: 0.9743 - categorical_accuracy: 0.69

Epoch 144/2000
3/3 [==============================] - 1s 170ms/step - loss: 1.4014 - categorical_accuracy: 0.2632
Epoch 145/2000
3/3 [==============================] - 1s 171ms/step - loss: 1.4009 - categorical_accuracy: 0.2632
Epoch 146/2000
3/3 [==============================] - 1s 174ms/step - loss: 1.4005 - categorical_accuracy: 0.2632
Epoch 147/2000
3/3 [==============================] - 0s 161ms/step - loss: 1.4003 - categorical_accuracy: 0.2632
Epoch 148/2000
3/3 [==============================] - 1s 167ms/step - loss: 1.3998 - categorical_accuracy: 0.2632
Epoch 149/2000
3/3 [==============================] - 1s 175ms/step - loss: 1.3995 - categorical_accuracy: 0.2632
Epoch 150/2000
3/3 [==============================] - 1s 183ms/step - loss: 1.3991 - categorical_accuracy: 0.2632
Epoch 151/2000
3/3 [==============================] - 1s 211ms/step - loss: 1.3988 - categorical_accuracy: 0.2632
Epoch 152/2000
3/3 [==============================] - 1s 179ms/step - loss: 1.3986 - cat

...

Epoch 859/2000
3/3 [==============================] - 1s 170ms/step - loss: 0.6584 - categorical_accuracy: 0.8289
Epoch 860/2000
3/3 [==============================] - 0s 159ms/step - loss: 0.7106 - categorical_accuracy: 0.7368
Epoch 861/2000
3/3 [==============================] - 1s 173ms/step - loss: 0.6671 - categorical_accuracy: 0.8289
Epoch 862/2000
3/3 [==============================] - 0s 158ms/step - loss: 0.6527 - categorical_accuracy: 0.8289
Epoch 863/2000
3/3 [==============================] - 0s 160ms/step - loss: 0.6721 - categorical_accuracy: 0.7632
Epoch 864/2000
3/3 [==============================] - 0s 167ms/step - loss: 0.6638 - categorical_accuracy: 0.7105
Epoch 865/2000
3/3 [==============================] - 0s 156ms/step - loss: 0.6715 - categorical_accuracy: 0.7500
Epoch 866/2000
3/3 [==============================] - 0s 156ms/step - loss: 0.6495 - categorical_accuracy: 0.7368
Epoch 867/2000
3/3 [==============================] - 0s 155ms/step - loss: 0.6548 - cat

3/3 [==============================] - 0s 159ms/step - loss: 0.5645 - categorical_accuracy: 0.8026
Epoch 931/2000
3/3 [==============================] - 1s 175ms/step - loss: 0.5682 - categorical_accuracy: 0.8026
Epoch 932/2000
3/3 [==============================] - 0s 156ms/step - loss: 0.5744 - categorical_accuracy: 0.8026
Epoch 933/2000
3/3 [==============================] - 1s 184ms/step - loss: 0.5792 - categorical_accuracy: 0.8421
Epoch 934/2000
3/3 [==============================] - 0s 160ms/step - loss: 0.5607 - categorical_accuracy: 0.8026
Epoch 935/2000
3/3 [==============================] - 0s 159ms/step - loss: 0.5807 - categorical_accuracy: 0.7763
Epoch 936/2000
3/3 [==============================] - 0s 161ms/step - loss: 0.5545 - categorical_accuracy: 0.7763
Epoch 937/2000
3/3 [==============================] - 0s 157ms/step - loss: 0.5449 - categorical_accuracy: 0.7763
Epoch 938/2000
3/3 [==============================] - 0s 163ms/step - loss: 0.5593 - categorical_accura

3/3 [==============================] - 0s 162ms/step - loss: 0.4415 - categorical_accuracy: 0.8816
Epoch 1002/2000
3/3 [==============================] - 0s 157ms/step - loss: 0.4231 - categorical_accuracy: 0.8684
Epoch 1003/2000
3/3 [==============================] - 0s 158ms/step - loss: 0.4144 - categorical_accuracy: 0.8684
Epoch 1004/2000
3/3 [==============================] - 0s 166ms/step - loss: 0.3981 - categorical_accuracy: 0.8816
Epoch 1005/2000
3/3 [==============================] - 0s 166ms/step - loss: 0.3959 - categorical_accuracy: 0.8816
Epoch 1006/2000
3/3 [==============================] - 0s 159ms/step - loss: 0.3894 - categorical_accuracy: 0.8947
Epoch 1007/2000
3/3 [==============================] - 0s 161ms/step - loss: 0.4329 - categorical_accuracy: 0.8684
Epoch 1008/2000
3/3 [==============================] - 0s 163ms/step - loss: 0.4166 - categorical_accuracy: 0.8553
Epoch 1009/2000
3/3 [==============================] - 0s 162ms/step - loss: 0.4330 - categorica

3/3 [==============================] - 1s 172ms/step - loss: 0.3093 - categorical_accuracy: 0.8816
Epoch 1073/2000
3/3 [==============================] - 0s 159ms/step - loss: 0.3251 - categorical_accuracy: 0.9079
Epoch 1074/2000
3/3 [==============================] - 0s 161ms/step - loss: 0.3028 - categorical_accuracy: 0.9211
Epoch 1075/2000
3/3 [==============================] - 0s 158ms/step - loss: 0.3022 - categorical_accuracy: 0.9211
Epoch 1076/2000
3/3 [==============================] - 0s 161ms/step - loss: 0.2834 - categorical_accuracy: 0.9079
Epoch 1077/2000
3/3 [==============================] - 0s 158ms/step - loss: 0.2960 - categorical_accuracy: 0.9079
Epoch 1078/2000
3/3 [==============================] - 0s 163ms/step - loss: 0.2937 - categorical_accuracy: 0.9211
Epoch 1079/2000
3/3 [==============================] - 1s 172ms/step - loss: 0.2846 - categorical_accuracy: 0.9211
Epoch 1080/2000
3/3 [==============================] - 0s 160ms/step - loss: 0.3099 - categorica

KeyboardInterrupt: 

In [19]:
model.save('action.h5')

In [21]:
model.load_weights('action.h5')

In [27]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

yhat = model.predict(X_train)

ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

multilabel_confusion_matrix(ytrue, yhat)

accuracy_score(ytrue, yhat)


3/3 [==============================] - 0s 23ms/step


0.9210526315789473